In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr
from groq import Groq

load_dotenv(override=True)

def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording {question}")
    return {"recorded": "ok"}

record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

class Me:
    def __init__(self):
        self.groq = Groq()
        self.name = "Manova"
        self.linkedin = ""
        self.summary = ""
        # Load LinkedIn text from PDF
        linkedin_pdf_path = "/home/iei/Desktop/agents/manova_ai_assistant/knowledge_base/AI-portfolio.pdf"
        if os.path.exists(linkedin_pdf_path):
            reader = PdfReader(linkedin_pdf_path)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    self.linkedin += text
        # Load summary text
        summary_path = "/home/iei/Desktop/agents/manova_ai_assistant/me/summary.txt"
        if os.path.exists(summary_path):
            with open(summary_path, "r", encoding="utf-8") as f:
                self.summary = f.read()

    def handle_tool_call(self, tool_calls):
        results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f"Tool called: {tool_name}", flush=True)
            tool = globals().get(tool_name)
            result = tool(**arguments) if tool else {}
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        return results
    
    def system_prompt(self):
        system_prompt = f"""You are acting as {self.name}, a professional AI assistant representing {self.name} on their personal website. 
Your responsibilities include:
1. Answering questions about {self.name}'s career, skills, experience, and background
2. Engaging professionally with potential clients or employers
3. Recording contact information when users want to connect
4. Reporting questions you can't answer

You have access to:
- A professional summary: {self.summary}
- LinkedIn profile content: {self.linkedin}

Guidelines:
- Be professional, friendly, and helpful
- Stay strictly on topic about {self.name}'s professional background
- If you don't know an answer, use the record_unknown_question tool
- When appropriate, encourage users to share their contact information using the record_user_details tool
- Never make up information about {self.name}"""
        return system_prompt
    
    def chat(self, message, history):
        # Strip extra keys from history to avoid Groq errors
        history = [{k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history]
        messages = [{"role": "system", "content": self.system_prompt()}] + history + [{"role": "user", "content": message}]
        done = False
        while not done:
            response = self.groq.chat.completions.create(
                model="llama-3.3-70b-versatile", 
                messages=messages, 
                tools=tools,
                temperature=0.7
            )
            if response.choices[0].finish_reason == "tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = self.handle_tool_call(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
        return response.choices[0].message.content

# Custom CSS for better aesthetics + contact buttons, accordion, etc.
custom_css = """
.gradio-container {
    font-family: 'Helvetica Neue', Arial, sans-serif;
    max-width: 800px;
    margin: 0 auto;
}
.dark .gradio-container {
    background: #1a1a1a;
}
.chatbot {
    min-height: 500px;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}
.dark .chatbot {
    background: #2d2d2d;
}
.textbox textarea {
    min-height: 80px !important;
    border-radius: 8px !important;
    padding: 12px !important;
}
.dark .textbox textarea {
    background: #333;
    color: white;
}
.button {
    border-radius: 8px !important;
    padding: 8px 16px !important;
    font-weight: 500 !important;
}
.dark .button {
    background: #4f46e5 !important;
}
.contact-buttons {
    display: flex;
    gap: 10px;
    margin-bottom: 20px;
    justify-content: center;
}
.contact-button {
    padding: 8px 16px;
    border-radius: 20px;
    text-decoration: none;
    display: flex;
    align-items: center;
    gap: 5px;
    font-size: 14px;
    transition: all 0.3s ease;
}
.contact-button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}
.email-button {
    background: #4285F4;
    color: white !important;
}
.linkedin-button {
    background: #0077B5;
    color: white !important;
}
.phone-button {
    background: #34B7F1;
    color: white !important;
}
.chat-input-container {
    display: flex;
    gap: 10px;
    align-items: center;
}
.contact-icon {
    font-size: 20px;
}
.accordion-header {
    font-weight: bold !important;
}
"""

if __name__ == "__main__":
    me = Me()
    
    with gr.Blocks(
        title=f"Chat with {me.name}",
        theme=gr.themes.Soft(),
        css=custom_css
    ) as demo:
        # Header with contact buttons
        gr.Markdown(f"""
        <div style='text-align: center; margin-bottom: 20px;'>
            <h1>Chat with {me.name}</h1>
            <div class='contact-buttons'>
                <a href='mailto:your.email@example.com' class='contact-button email-button'>
                    <span class='contact-icon'>✉️</span> Email
                </a>
                <a href='https://linkedin.com/in/yourprofile' target='_blank' class='contact-button linkedin-button'>
                    <span class='contact-icon'>🔗</span> LinkedIn
                </a>
                <a href='tel:+1234567890' class='contact-button phone-button'>
                    <span class='contact-icon'>📞</span> Call
                </a>
            </div>
        </div>
        <p style='text-align: center; margin-bottom: 20px;'>
            Ask me about my professional background, skills, and experience.
            I'll do my best to answer your questions about my career and work.
        </p>
        """)
        
        chatbot = gr.Chatbot(
            bubble_full_width=False,
            show_copy_button=True,
            avatar_images=("user.png", "bot.png"),
            height=500
        )
        
        with gr.Row():
            textbox = gr.Textbox(
                placeholder="Type your message here...",
                container=False,
                autofocus=True,
                lines=2,
                scale=7
            )
            submit_btn = gr.Button("Send", variant="primary", scale=1)
        
        # Contact info inputs with icons
        with gr.Accordion("📩 Contact Information (optional)", open=False):
            with gr.Row():
                name_input = gr.Textbox(label="Your Name", lines=1)
                email_input = gr.Textbox(label="Your Email", lines=1)
        
        # Example questions
        gr.Examples(
            examples=[
                "What's your professional background?",
                "What skills do you have?",
                "Can you tell me about your work experience?",
                "How can I contact you for potential opportunities?"
            ],
            inputs=textbox,
            label="💡 Example Questions"
        )
        
        def respond(message, chat_history, name, email):
            # Convert chat history to Groq format
            groq_history = []
            for user_msg, bot_msg in chat_history:
                groq_history.append({"role": "user", "content": user_msg})
                groq_history.append({"role": "assistant", "content": bot_msg})
            
            response = me.chat(message, groq_history)
            chat_history.append((message, response))
            
            if email:
                record_user_details(email=email, name=name, notes=f"From chat: {message}")
            
            return chat_history
        
        submit_btn.click(
            respond,
            inputs=[textbox, chatbot, name_input, email_input],
            outputs=[chatbot],
            queue=False
        ).then(
            lambda: "",
            outputs=[textbox]
        )
        
        textbox.submit(
            respond,
            inputs=[textbox, chatbot, name_input, email_input],
            outputs=[chatbot],
            queue=False
        ).then(
            lambda: "",
            outputs=[textbox]
        )
    
    demo.launch(
        server_name="0.0.0.0",
        server_port=7862,
        share=False,
        favicon_path="favicon.ico"
    )


In [ ]:
import os
import json
from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr
from groq import Groq
from PIL import Image
import pytesseract
from datetime import datetime
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# Try importing FAISS
try:
    from langchain.vectorstores import FAISS
    FAISS_AVAILABLE = True
except ImportError:
    FAISS_AVAILABLE = False

load_dotenv(override=True)

# Contact Information - REPLACE WITH YOUR ACTUAL INFO
YOUR_EMAIL = "your.email@example.com"
YOUR_PHONE = "+1234567890"
YOUR_LINKEDIN = "https://linkedin.com/in/yourprofile"
YOUR_GITHUB = "https://github.com/yourprofile"

# Custom CSS for professional look
PROFESSIONAL_CSS = """
:root {
    --primary: #6366f1;
    --primary-dark: #4f46e5;
    --secondary: #f8fafc;
    --accent: #10b981;
    --text: #1e293b;
    --light-text: #64748b;
    --card-bg: #ffffff;
}
.gradio-container {
    font-family: 'Inter', 'Segoe UI', Roboto, sans-serif;
    max-width: 900px;
    margin: 2rem auto;
    background: var(--secondary);
    border-radius: 16px;
    box-shadow: 0 10px 25px rgba(0,0,0,0.05);
}
.header-card {
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-dark) 100%);
    color: white;
    padding: 2.5rem 2rem;
    border-radius: 16px 16px 0 0;
    text-align: center;
    position: relative;
    overflow: hidden;
}
.header-card::before {
    content: "";
    position: absolute;
    top: -50px;
    right: -50px;
    width: 200px;
    height: 200px;
    background: radial-gradient(circle, rgba(255,255,255,0.1) 0%, transparent 70%);
    border-radius: 50%;
}
.header-content {
    position: relative;
    z-index: 2;
}
.contact-buttons {
    display: flex;
    gap: 12px;
    justify-content: center;
    margin-top: 1.5rem;
    flex-wrap: wrap;
}
.contact-button {
    padding: 10px 20px;
    border-radius: 8px;
    text-decoration: none;
    display: flex;
    align-items: center;
    gap: 8px;
    font-weight: 500;
    transition: all 0.3s ease;
    font-size: 0.9rem;
    border: none;
    cursor: pointer;
}
.contact-button:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 16px rgba(0,0,0,0.15);
}
.email-button { background: #ea4335; color: white !important; }
.linkedin-button { background: #0a66c2; color: white !important; }
.phone-button { background: #34a853; color: white !important; }
.github-button { background: #333; color: white !important; }
.chat-container {
    display: flex;
    flex-direction: column;
    height: 100%;
    background: var(--card-bg);
    border-radius: 0 0 16px 16px;
    overflow: hidden;
    box-shadow: 0 4px 12px rgba(0,0,0,0.05);
}
.chatbot {
    min-height: 500px;
    max-height: 600px;
    overflow-y: auto;
    padding: 1.5rem;
    background: var(--card-bg);
    border-radius: 0;
    scrollbar-width: thin;
    scrollbar-color: var(--primary) var(--secondary);
}
.chatbot::-webkit-scrollbar {
    width: 6px;
}
.chatbot::-webkit-scrollbar-track {
    background: var(--secondary);
}
.chatbot::-webkit-scrollbar-thumb {
    background-color: var(--primary);
    border-radius: 6px;
}
.textbox {
    border-radius: 12px !important;
    padding: 14px 16px !important;
    border: 1px solid #e2e8f0 !important;
    background: var(--card-bg) !important;
    box-shadow: none !important;
}
.textbox:focus {
    border-color: var(--primary) !important;
    box-shadow: 0 0 0 2px rgba(99,102,241,0.2) !important;
}
.submit-button {
    background: var(--primary) !important;
    color: white !important;
    border-radius: 12px !important;
    padding: 0 24px !important;
    height: 100% !important;
    transition: all 0.3s ease !important;
}
.submit-button:hover {
    background: var(--primary-dark) !important;
    transform: translateY(-2px);
    box-shadow: 0 4px 12px rgba(99,102,241,0.3) !important;
}
.quick-questions {
    display: flex;
    gap: 10px;
    flex-wrap: wrap;
    margin-bottom: 1rem;
}
.quick-question {
    background: var(--secondary);
    border: 1px solid #e2e8f0;
    border-radius: 20px;
    padding: 8px 16px;
    font-size: 0.85rem;
    color: var(--light-text);
    cursor: pointer;
    transition: all 0.2s ease;
}
.quick-question:hover {
    background: var(--primary);
    color: white;
    border-color: var(--primary);
}
.user-message {
    background: var(--secondary);
    border-radius: 12px 12px 0 12px !important;
    padding: 12px 16px;
    max-width: 80%;
    margin-left: auto;
    border: 1px solid #e2e8f0;
}
.bot-message {
    background: var(--primary);
    color: white;
    border-radius: 12px 12px 12px 0 !important;
    padding: 12px 16px;
    max-width: 80%;
    margin-right: auto;
}
.accordion {
    background: var(--card-bg) !important;
    border: 1px solid #e2e8f0 !important;
    border-radius: 12px !important;
    margin: 1rem 0 !important;
}
.accordion-item {
    border: none !important;
}
.stats-card {
    background: var(--card-bg);
    border-radius: 12px;
    padding: 1.5rem;
    margin: 1rem 0;
    border: 1px solid #e2e8f0;
}
.stats-row {
    display: flex;
    justify-content: space-between;
    margin-bottom: 1rem;
}
.stat-item {
    text-align: center;
    padding: 0 1rem;
}
.stat-value {
    font-size: 1.5rem;
    font-weight: 600;
    color: var(--primary);
}
.stat-label {
    font-size: 0.8rem;
    color: var(--light-text);
    margin-top: 0.25rem;
}
.tabs {
    margin: 1rem 0;
}
.tab-button {
    padding: 10px 20px !important;
    border-radius: 8px 8px 0 0 !important;
}
"""

class Me:
    def __init__(self):
        self.groq = Groq()
        self.name = "Manova"
        self.title = "AI & Data Science Specialist"
        self.description = "Digital assistant with expertise in artificial intelligence, machine learning, and data analysis. Ready to help with your technical questions and professional inquiries."
        self.knowledge_base = ""
        self.chat_history = []
        self._load_documents()
        self._initialize_rag()
        self.interaction_count = 0
        self.start_time = datetime.now()

    def _load_documents(self):
        """Load all documents from knowledge_base folder"""
        doc_paths = {
            "pdf": ["me/cv.pdf", "knowledge_base/AI-portfolio.pdf.pdf"],
            "text": ["me/summary.txt"],
            "images": ["/home/iei/Desktop/agents/manova_ai_assistant/knowledge_base/certifications/WhatsApp Image 2024-04-18 at 2.59.18 PM (4).jpeg"]
        }
        
        for path in doc_paths["pdf"]:
            if os.path.exists(path):
                reader = PdfReader(path)
                for page in reader.pages:
                    if text := page.extract_text():
                        self.knowledge_base += f"\n\n{text}"
        
        for path in doc_paths["text"]:
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    self.knowledge_base += f"\n\n{f.read()}"
        
        for path in doc_paths["images"]:
            if os.path.exists(path):
                self.knowledge_base += f"\n\n{pytesseract.image_to_string(Image.open(path))}"

    def _initialize_rag(self):
        """Initialize RAG components"""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        self.text_chunks = text_splitter.split_text(self.knowledge_base)
        
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        
        if FAISS_AVAILABLE:
            self.vector_db = FAISS.from_texts(self.text_chunks, embeddings)
        else:
            self.vector_db = None

    def search_documents(self, query):
        """Search knowledge base"""
        if self.vector_db:
            docs = self.vector_db.similarity_search(query, k=3)
            return "\n\n".join([doc.page_content for doc in docs])
        return self.knowledge_base[:2000]  # Fallback

    def system_prompt(self):
        return f"""You are {self.name}, {self.title}. {self.description}

        About me:
        {self.knowledge_base[:10000]}

        Guidelines:
        - Be concise, professional, and knowledgeable
        - Maintain a friendly but expert tone
        - Offer to connect via contact buttons when appropriate
        - Never make up information
        - For technical questions, provide detailed explanations
        - For requests outside your knowledge, politely decline
        - Current date: {datetime.now().strftime('%Y-%m-%d')}"""

    def chat(self, message, history):
        self.interaction_count += 1
        context = self.search_documents(message)
        messages = [
            {"role": "system", "content": self.system_prompt()},
            *history,
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {message}"}
        ]
        response = self.groq.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
            temperature=0.7
        )
        return response.choices[0].message.content

def create_interface():
    me = Me()
    
    # Predefined questions
    quick_questions = [
        "What's your experience with AI?",
        "Can you summarize your skills?",
        "What projects have you worked on?",
        "How can I contact you?",
        "What technologies do you specialize in?"
    ]
    
    with gr.Blocks(
        title=f"{me.name} | {me.title}",
        css=PROFESSIONAL_CSS,
        theme=gr.themes.Soft(
            primary_hue="indigo",
            secondary_hue="slate"
        )
    ) as demo:
        # Header Card with Contact Buttons
        with gr.Column(elem_classes="header-card"):
            with gr.Column(elem_classes="header-content"):
                gr.Markdown(f"# {me.name}")
                gr.Markdown(f"### {me.title}")
                gr.Markdown(me.description)
                
                with gr.Row(elem_classes="contact-buttons"):
                    gr.HTML(f"""
                    <a href="mailto:{YOUR_EMAIL}" class="contact-button email-button">
                        <span>✉️</span> Email
                    </a>
                    <a href="{YOUR_LINKEDIN}" target="_blank" class="contact-button linkedin-button">
                        <span>🔗</span> LinkedIn
                    </a>
                    <a href="tel:{YOUR_PHONE}" class="contact-button phone-button">
                        <span>📞</span> Call
                    </a>
                    <a href="{YOUR_GITHUB}" target="_blank" class="contact-button github-button">
                        <span>💻</span> GitHub
                    </a>
                    """)
        
        # Stats Card
        with gr.Column(visible=True) as stats_col:
            with gr.Column(elem_classes="stats-card"):
                gr.Markdown("### Interaction Stats")
                with gr.Row(elem_classes="stats-row"):
                    with gr.Column(elem_classes="stat-item"):
                        interaction_count_text = gr.Markdown(f"<div class='stat-value'>{me.interaction_count}</div>", elem_classes="stat-value")
                        gr.Markdown("<div class='stat-label'>Questions Answered</div>", elem_classes="stat-label")
                    with gr.Column(elem_classes="stat-item"):
                        uptime = (datetime.now() - me.start_time).seconds // 60
                        uptime_text = gr.Markdown(f"<div class='stat-value'>{uptime}</div>", elem_classes="stat-value")
                        gr.Markdown("<div class='stat-label'>Minutes Active</div>", elem_classes="stat-label")
                    with gr.Column(elem_classes="stat-item"):
                        gr.Markdown(f"<div class='stat-value'>Llama3</div>", elem_classes="stat-value")
                        gr.Markdown("<div class='stat-label'>AI Model</div>", elem_classes="stat-label")
        
        # Main Chat Interface
        with gr.Column(elem_classes="chat-container"):
            msg = gr.Textbox(
                placeholder="Type your question here...",
                lines=2,
                elem_classes="textbox",
                interactive=True
            )
            chatbot = gr.Chatbot(elem_classes="chatbot", show_copy_button=True,
                                 bubble_full_width=False, render_markdown=True)
            submit_btn = gr.Button("Send", elem_classes="submit-button")
            
            # Quick questions buttons
            with gr.Row(elem_classes="quick-questions"):
                for q in quick_questions:
                    btn = gr.Button(q, elem_classes="quick-question")
                    btn.click(lambda question=q: question, inputs=None, outputs=msg)
            
            def respond(message, chat_history):
                history_formatted = []
                for user_msg, bot_msg in chat_history:
                    history_formatted.append({"role": "user", "content": user_msg})
                    history_formatted.append({"role": "assistant", "content": bot_msg})
                response = me.chat(message, history_formatted)
                chat_history.append((message, response))
                # Update stats
                interaction_count_text.update(value=f"<div class='stat-value'>{me.interaction_count}</div>")
                uptime_val = (datetime.now() - me.start_time).seconds // 60
                uptime_text.update(value=f"<div class='stat-value'>{uptime_val}</div>")
                return chat_history
            
            submit_btn.click(
                respond,
                inputs=[msg, chatbot],
                outputs=[chatbot],
                queue=True
            )
            msg.submit(
                respond,
                inputs=[msg, chatbot],
                outputs=[chatbot],
                queue=True
            )
            
    return demo

if __name__ == "__main__":
    demo = create_interface()
    demo.launch(server_name="0.0.0.0", server_port=7865, share=False)


In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr
from groq import Groq

load_dotenv(override=True)

def push(text):
    requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": os.getenv("PUSHOVER_TOKEN"),
            "user": os.getenv("PUSHOVER_USER"),
            "message": text,
        }
    )

def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording unknown question: {question}")
    return {"recorded": "ok"}

record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {"type": "string", "description": "The email address of this user"},
            "name": {"type": "string", "description": "The user's name, if they provided it"},
            "notes": {"type": "string", "description": "Additional context or notes about the conversation"}
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {"type": "string", "description": "The question that couldn't be answered"}
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]

class Me:
    def __init__(self):
        self.groq = Groq()
        self.name = "Manova"
        self.knowledge_base_text = ""
        self.image_paths = []
        self._load_documents()

    def _load_documents(self):
        """Load documents from multiple formats and paths"""
        doc_paths = {
            "pdf": [
                "me/cv.pdf",
                "knowledge_base/AI-portfolio.pdf.pdf"
            ],
            "text": [
                "me/summary.txt"
            ],
            "images": [
                "/home/iei/Desktop/agents/manova_ai_assistant/knowledge_base/certifications/WhatsApp Image 2024-04-18 at 2.59.18 PM (4).jpeg"
            ]
        }

        self.knowledge_base_text = ""  # Reset before loading

        # Load PDF text
        for pdf_path in doc_paths["pdf"]:
            if os.path.exists(pdf_path):
                print(f"Loading PDF: {pdf_path}")
                try:
                    reader = PdfReader(pdf_path)
                    for i, page in enumerate(reader.pages):
                        text = page.extract_text()
                        if text:
                            self.knowledge_base_text += "\n" + text
                            print(f"Loaded page {i+1} of {pdf_path} with {len(text)} chars")
                        else:
                            print(f"No text found on page {i+1} of {pdf_path}")
                except Exception as e:
                    print(f"Failed to read PDF {pdf_path}: {e}")
            else:
                print(f"PDF not found: {pdf_path}")

        # Load text files
        for txt_path in doc_paths["text"]:
            if os.path.exists(txt_path):
                print(f"Loading text file: {txt_path}")
                try:
                    with open(txt_path, "r", encoding="utf-8") as f:
                        content = f.read()
                        self.knowledge_base_text += "\n" + content
                        print(f"Loaded text file {txt_path} with {len(content)} chars")
                except Exception as e:
                    print(f"Failed to read text file {txt_path}: {e}")
            else:
                print(f"Text file not found: {txt_path}")

        # Store image paths (just existence check)
        self.image_paths = [p for p in doc_paths["images"] if os.path.exists(p)]
        print(f"Loaded {len(self.image_paths)} image paths")

        print(f"Total knowledge base length: {len(self.knowledge_base_text)} characters")

    def handle_tool_call(self, tool_calls):
        results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f"Tool called: {tool_name}", flush=True)
            tool = globals().get(tool_name)
            result = tool(**arguments) if tool else {}
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        return results

    def system_prompt(self):
        return f"""You are acting as {self.name}, a professional AI assistant representing {self.name} on their personal website.
Your responsibilities include:
- Answering questions ONLY based on the knowledge base below. Do NOT answer if the information is not contained in the knowledge base.
- If you do NOT know the answer from the knowledge base, respond: "I don't know" and record the question.
- Engage professionally and politely.
- Use tools to record unknown questions and user details.
- NEVER make up information or guess beyond the knowledge base.

Knowledge Base Content:
{self.knowledge_base_text}

Guidelines:
- Be professional, friendly, and helpful.
- Stay strictly on topic about {self.name}'s professional background."""

    def chat(self, message, history):
        # Clean history keys to avoid Groq errors
        history = [{k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history]
        messages = [{"role": "system", "content": self.system_prompt()}] + history + [{"role": "user", "content": message}]
        done = False
        recorded_unknown = False
        while not done:
            response = self.groq.chat.completions.create(
                model="llama-3.3-70b-versatile", 
                messages=messages, 
                tools=tools,
                temperature=0.7
            )
            if response.choices[0].finish_reason == "tool_calls":
                message_obj = response.choices[0].message
                tool_calls = message_obj.tool_calls
                results = self.handle_tool_call(tool_calls)
                messages.append(message_obj)
                messages.extend(results)
            else:
                done = True

        answer = response.choices[0].message.content

        # Check if answer indicates lack of knowledge, then record unknown question
        if any(phrase in answer.lower() for phrase in ["i don't know", "cannot answer", "not in my knowledge base", "no information"]):
            if not recorded_unknown:
                record_unknown_question(question=message)
                recorded_unknown = True

        return answer

# Custom CSS for UI styling
custom_css = """
.gradio-container {
    font-family: 'Helvetica Neue', Arial, sans-serif;
    max-width: 800px;
    margin: 0 auto;
}
.dark .gradio-container {
    background: #1a1a1a;
}
.chatbot {
    min-height: 500px;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}
.dark .chatbot {
    background: #2d2d2d;
}
.textbox textarea {
    min-height: 80px !important;
    border-radius: 8px !important;
    padding: 12px !important;
}
.dark .textbox textarea {
    background: #333;
    color: white;
}
.button {
    border-radius: 8px !important;
    padding: 8px 16px !important;
    font-weight: 500 !important;
}
.dark .button {
    background: #4f46e5 !important;
}
.contact-buttons {
    display: flex;
    gap: 10px;
    margin-bottom: 20px;
    justify-content: center;
}
.contact-button {
    padding: 8px 16px;
    border-radius: 20px;
    text-decoration: none;
    display: flex;
    align-items: center;
    gap: 5px;
    font-size: 14px;
    transition: all 0.3s ease;
}
.contact-button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}
.email-button {
    background: #4285F4;
    color: white !important;
}
.linkedin-button {
    background: #0077B5;
    color: white !important;
}
.phone-button {
    background: #34B7F1;
    color: white !important;
}
.chat-input-container {
    display: flex;
    gap: 10px;
    align-items: center;
}
.contact-icon {
    font-size: 20px;
}
.accordion-header {
    font-weight: bold !important;
}
"""

if __name__ == "__main__":
    me = Me()

    with gr.Blocks(
        title=f"Chat with {me.name}",
        theme=gr.themes.Soft(),
        css=custom_css
    ) as demo:
        # Header with contact buttons
        gr.Markdown(f"""
        <div style='text-align: center; margin-bottom: 20px;'>
            <h1>Chat with {me.name}</h1>
            <div class='contact-buttons'>
                <a href='mailto:your.email@example.com' class='contact-button email-button'>
                    <span class='contact-icon'>✉️</span> Email
                </a>
                <a href='https://linkedin.com/in/yourprofile' target='_blank' class='contact-button linkedin-button'>
                    <span class='contact-icon'>🔗</span> LinkedIn
                </a>
                <a href='tel:+1234567890' class='contact-button phone-button'>
                    <span class='contact-icon'>📞</span> Call
                </a>
            </div>
        </div>
        <p style='text-align: center; margin-bottom: 20px;'>
            Ask me about my professional background, skills, and experience.
            I'll do my best to answer your questions about my career and work.
        </p>
        """)

        chatbot = gr.Chatbot(
            bubble_full_width=False,
            show_copy_button=True,
            avatar_images=("user.png", "bot.png"),
            height=500
        )

        with gr.Row():
            textbox = gr.Textbox(
                placeholder="Type your message here...",
                container=False,
                autofocus=True,
                lines=2,
                scale=7
            )
            submit_btn = gr.Button("Send", variant="primary", scale=1)

        with gr.Accordion("📩 Contact Information (optional)", open=False):
            with gr.Row():
                name_input = gr.Textbox(label="Your Name", lines=1)
                email_input = gr.Textbox(label="Your Email", lines=1)

        gr.Examples(
            examples=[
                "What's your professional background?",
                "What skills do you have?",
                "Can you tell me about your work experience?",
                "How can I contact you for potential opportunities?"
            ],
            inputs=textbox,
            label="💡 Example Questions"
        )

        def respond(message, chat_history, name, email):
            groq_history = []
            for user_msg, bot_msg in chat_history:
                groq_history.append({"role": "user", "content": user_msg})
                groq_history.append({"role": "assistant", "content": bot_msg})

            response = me.chat(message, groq_history)
            chat_history.append((message, response))

            if email:
                record_user_details(email=email, name=name or "Name not provided", notes=f"From chat: {message}")

            return chat_history

        submit_btn.click(
            respond,
            inputs=[textbox, chatbot, name_input, email_input],
            outputs=[chatbot],
            queue=False
        ).then(
            lambda: "",
            outputs=[textbox]
        )

        textbox.submit(
            respond,
            inputs=[textbox, chatbot, name_input, email_input],
            outputs=[chatbot],
            queue=False
        ).then(
            lambda: "",
            outputs=[textbox]
        )

    demo.launch(
        server_name="0.0.0.0",
        server_port=7863,
        share=False,
        favicon_path="favicon.ico"
    )


/home/iei/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1227640/136830455.py:294: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipykernel_1227640/136830455.py:294: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


PDF not found: me/cv.pdf
PDF not found: knowledge_base/AI-portfolio.pdf.pdf
Text file not found: me/summary.txt
Loaded 1 image paths
Total knowledge base length: 0 characters
* Running on local URL:  http://0.0.0.0:7863
* To create a public link, set `share=True` in `launch()`.
